In [1]:
import warnings
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [2]:
firms = pd.read_csv("../datasets/dont_save/base_etablissement_par_tranche_effectif.csv")
geography = pd.read_csv("../datasets/dont_save/name_geographic_information.csv")
salary = pd.read_csv("../datasets/dont_save/net_salary_per_town_categories.csv")
population = pd.read_csv("../datasets/dont_save/population.csv")

In [3]:
geography.drop_duplicates(subset=["code_insee"], keep="first", inplace=True)

In [4]:
firms["CODGEO"] = firms["CODGEO"].astype(str)
geography['code_insee'] = geography['code_insee'].astype(str)
salary["CODGEO"] = salary["CODGEO"].astype(str)
population["CODGEO"] = population["CODGEO"].astype(str)
merge1 = firms.merge(geography,left_on= "CODGEO",right_on="code_insee")
merge2 = merge1.merge(salary, on= "CODGEO")
merge3 = merge2.merge(population, on= "CODGEO")

In [5]:
data = merge3.drop(columns=['LIBGEO_x', 'REG', 'DEP','EU_circo', 'code_région', 'chef.lieu_région', 'préfecture','numéro_circonscription','codes_postaux', 'code_insee',
                            'éloignement', 'LIBGEO_y','NIVGEO','LIBGEO'])

In [18]:
groupby_columns = list(data.columns[:-4])
data2 = data.reset_index().groupby(groupby_columns, as_index= False).sum()

In [19]:
data2 = data2.drop(columns = ['index', 'MOCO', 'AGEQ80_17','SEXE'])

In [20]:
data2['Micro_firms'] = data2['E14TS1'] + data2['E14TS6']
data2['Small_firms'] = data2['E14TS10'] + data2['E14TS20']
data2['Medium_firms'] = data2['E14TS50'] + data2['E14TS100']
data2['Large_firms'] = data2['E14TS200'] + data2['E14TS500']
data2['Departement'] =' (' + data2['numéro_département'] + ') ' + data2['nom_département'] 
data2 = data2.drop(columns=['nom_département', 'numéro_département', 'E14TS0ND', 'E14TS1', 'E14TS6', 'E14TS10','E14TS20', 'E14TS50', 'E14TS100', 'E14TS200', 'E14TS500','E14TS0ND'])
data2 = data2.rename(columns = {'E14TST':'total_firms','NB':'total_population','nom_commune':'Town'})

In [21]:
data2.head()

,CODGEO,total_firms,nom_région,Town,latitude,longitude,SNHM14,SNHMC14,SNHMP14,SNHME14,...,SNHMF5014,SNHMH1814,SNHMH2614,SNHMH5014,total_population,Micro_firms,Small_firms,Medium_firms,Large_firms,Departement
0,10003,140,Champagne-Ardenne,Aix-en-Othe,48.216667,3.733333,11.2,20.2,12.8,9.5,...,10.5,9.7,11.7,13.3,3440,42,5,0,0,(10) Aube
1,10006,203,Champagne-Ardenne,Arcis-sur-Aube,48.533333,4.133333,11.6,19.3,14.8,9.6,...,11.8,9.6,11.9,13.7,2708,73,15,2,0,(10) Aube
2,10033,397,Champagne-Ardenne,Bar-sur-Aube,48.233333,4.716667,11.6,21.4,14.1,10.2,...,10.9,9.7,11.9,14.7,4704,154,24,6,1,(10) Aube
3,10034,273,Champagne-Ardenne,Bar-sur-Seine,48.116667,4.366667,11.4,24.2,12.7,9.5,...,11.1,9.3,11.1,15.8,2785,106,20,3,0,(10) Aube
4,10060,147,Champagne-Ardenne,Bréviandes,48.250000,4.1,13.4,23.0,15.5,10.5,...,12.5,10.1,14.2,16.4,2499,46,19,2,0,(10) Aube


In [22]:
data2=data2.rename(columns = {'SNHM14':'mean_salary',
                              'SNHMC14':'mean_executive_salary',
                              'SNHMP14':'mean_middle_manager_salary',
                              'SNHME14':'mean_employee_salary',
                              'SNHMO14':'mean_worker_salary',
                              'SNHMF14':'mean_female_salary',
                              'SNHMFC14':'mean_female_executive_salary',
                              'SNHMFP14':'mean_female_middle_manager_salary',
                              'SNHMFE14':'mean_female_employee_salary',
                              'SNHMFO14':'mean_female_worker_salary',
                              'SNHMH14':'mean_male_salary',
                              'SNHMHC14':'mean_male_executive_salary',
                              'SNHMHP14':'mean_male_middle_manager_salary',
                              'SNHMHE14':'mean_male_employee_salary',
                              'SNHMHO14':'mean_male_worker_salary',
                              'SNHM1814':'mean_young_age_salary',
                              'SNHM2614':'mean_medium_age_salary',
                              'SNHM5014':'mean_old_age_salary',
                              'SNHMF1814':'mean_young_female_salary',
                              'SNHMF2614':'mean_medium_female_salary',
                              'SNHMF5014':'mean_old_female_salary',
                              'SNHMH1814':'mean_young_male_salary',
                              'SNHMH2614':'mean_medium_male_salary',
                              'SNHMH5014':'mean_old_male_salary',
                              })

In [23]:
data2['longitude'] = data2['longitude'].replace('-', 0).astype(float)

In [24]:
data2

,CODGEO,total_firms,nom_région,Town,latitude,longitude,mean_salary,mean_executive_salary,mean_middle_manager_salary,mean_employee_salary,...,mean_old_female_salary,mean_young_male_salary,mean_medium_male_salary,mean_old_male_salary,total_population,Micro_firms,Small_firms,Medium_firms,Large_firms,Departement
0,10003,140,Champagne-Ardenne,Aix-en-Othe,48.216667,3.733333,11.2,20.2,12.8,9.5,...,10.5,9.7,11.7,13.3,3440,42,5,0,0,(10) Aube
1,10006,203,Champagne-Ardenne,Arcis-sur-Aube,48.533333,4.133333,11.6,19.3,14.8,9.6,...,11.8,9.6,11.9,13.7,2708,73,15,2,0,(10) Aube
2,10033,397,Champagne-Ardenne,Bar-sur-Aube,48.233333,4.716667,11.6,21.4,14.1,10.2,...,10.9,9.7,11.9,14.7,4704,154,24,6,1,(10) Aube
3,10034,273,Champagne-Ardenne,Bar-sur-Seine,48.116667,4.366667,11.4,24.2,12.7,9.5,...,11.1,9.3,11.1,15.8,2785,106,20,3,0,(10) Aube
4,10060,147,Champagne-Ardenne,Bréviandes,48.250000,4.100000,13.4,23.0,15.5,10.5,...,12.5,10.1,14.2,16.4,2499,46,19,2,0,(10) Aube
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4397,95612,443,Île-de-France,Le Thillay,49.000000,2.466667,13.7,22.1,16.0,11.3,...,14.6,10.2,14.0,17.1,4080,135,40,5,0,(95) Val-d'Oise
4398,95637,551,Île-de-France,Vauréal,49.033333,2.033333,16.3,25.3,15.4,11.7,...,15.4,10.2,16.8,22.5,15754,104,8,0,0,(95) Val-d'Oise
4399,95641,113,Île-de-France,Vémars,49.066667,2.566667,16.3,24.7,16.5,12.3,...,16.6,10.4,17.3,20.2,2290,14,6,4,0,(95) Val-d'Oise
4400,95652,309,Île-de-France,Viarmes,49.133333,2.366667,16.7,26.9,16.0,12.0,...,16.8,10.6,18.1,21.7,5070,89,9,0,0,(95) Val-d'Oise


In [25]:
data2.to_csv("final_data.csv")